<a href="https://colab.research.google.com/github/dpcks/DACON--AI-/blob/main/CNN%2BTFIDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import

In [ ]:
import random
import pandas as pd
import numpy as np
import os
from PIL import Image
from sklearn.preprocessing import LabelEncoder # 정답값을 정수로 바꿔야하니깐 레이블 인코딩
from sklearn.feature_extraction.text import CountVectorizer ,TfidfVectorizer # tf가 성능 좀더오름

import torch

from tqdm.auto import tqdm
from sklearn.metrics import f1_score

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
DATA_PATH = "/content/drive/MyDrive/dacon/poi/data/"
SEED = 42

In [ ]:
if not os.path.isdir("image"): # 압축을 풀게 되면 이미지라는 디렉토리가 생김
    !cp "{DATA_PATH}open.zip" "open.zip"
    !unzip -qq "open.zip"

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [ ]:
train # 문맥의 순서보다는 어떤 단어가 들어가는지가 더중요하지 않을까?

,id,img_path,overview,cat1,cat2,cat3
0,TRAIN_00000,./image/train/TRAIN_00000.jpg,소안항은 조용한 섬으로 인근해안이 청정해역으로 일찍이 김 양식을 해서 높은 소득을 ...,자연,자연관광지,항구/포구
1,TRAIN_00001,./image/train/TRAIN_00001.jpg,경기도 이천시 모가면에 있는 골프장으로 대중제 18홀이다. 회원제로 개장을 했다가 ...,레포츠,육상 레포츠,골프
2,TRAIN_00002,./image/train/TRAIN_00002.jpg,금오산성숯불갈비는 한우고기만을 전문적으로 취급하고 사용하는 부식 자재 또한 유기농법...,음식,음식점,한식
3,TRAIN_00003,./image/train/TRAIN_00003.jpg,철판 위에서 요리하는 안동찜닭을 맛볼 수 있는 곳이다. 경상북도 안동시에 있는 한식...,음식,음식점,한식
4,TRAIN_00004,./image/train/TRAIN_00004.jpg,※ 영업시간 10:30 ~ 20:30\n\n3대에 걸쳐 아귀만을 전문으로 취급하는 ...,음식,음식점,한식
...,...,...,...,...,...,...
16981,TRAIN_16981,./image/train/TRAIN_16981.jpg,해발 12000m에 자리한 식담겸 카페점문점이다.<br>곤드레밥과 감자전을 판매하고...,음식,음식점,한식
16982,TRAIN_16982,./image/train/TRAIN_16982.jpg,설악힐호텔은 동해고속도로 속초톨게이트에서 멀지 않은 관광로 변에 있다. 속초의 대표...,숙박,숙박시설,모텔
16983,TRAIN_16983,./image/train/TRAIN_16983.jpg,충남 서산시 중심가에 위치한 줌모텔은 프라이버스가 보장되는 조용한 공간으로 가치가 ...,숙박,숙박시설,모텔
16984,TRAIN_16984,./image/train/TRAIN_16984.jpg,토토큰바위캠핑장은 경기도 가평지역 내에서도 청정지역으로 손꼽히는 지역으로 주변에 화...,레포츠,육상 레포츠,"야영장,오토캠핑장"


In [ ]:
test

,id,img_path,overview
0,TEST_00000,./image/test/TEST_00000.jpg,신선한 재료로 정성을 다해 만들었다. 늘 변함없는 맛과 서비스로 모실것을 약속한다.
1,TEST_00001,./image/test/TEST_00001.jpg,"청청한 해역 등량만과 율포해수욕장이 한눈에 내려다 보이는 위치에 있으며, 막 잡은 ..."
2,TEST_00002,./image/test/TEST_00002.jpg,장터설렁탕은 남녀노소 누구나 즐길 수 있는 전통 건강식으로 좋은 재료와 전통 조리방...
3,TEST_00003,./image/test/TEST_00003.jpg,다양한 형태의 청소년수련활동을 제공함으로써 청소년들이 민주사회의 주역이 될 수 있도...
4,TEST_00004,./image/test/TEST_00004.jpg,팔공산은 경산시의 북쪽에 위치한 해발 1192.3 m의 높은 산으로 신라시대에는 중...
...,...,...,...
7275,TEST_07275,./image/test/TEST_07275.jpg,"막국수와 수육을 주메뉴로 하며, 넓은 주차장이 마련되어 있어 주차하기 편리하다.<br>"
7276,TEST_07276,./image/test/TEST_07276.jpg,"통진두레문화센터는 우리고유의 전통무형문화와 민속예술을 계승/발전 시키고, 다양한 문..."
7277,TEST_07277,./image/test/TEST_07277.jpg,"수도권에서 가까운 위치, 문산천을 따라 걷는 산책코스, 한여름 더위를 날려버릴 시원..."
7278,TEST_07278,./image/test/TEST_07278.jpg,\n전남 구례군 관산리에 위치한 노고단 게스트하우스&호텔은 지리산을 파노라마로 관망...


In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16986 entries, 0 to 16985
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        16986 non-null  object
 1   img_path  16986 non-null  object
 2   overview  16986 non-null  object
 3   cat1      16986 non-null  object
 4   cat2      16986 non-null  object
 5   cat3      16986 non-null  object
dtypes: object(6)
memory usage: 796.3+ KB


In [ ]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7280 entries, 0 to 7279
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7280 non-null   object
 1   img_path  7280 non-null   object
 2   overview  7280 non-null   object
dtypes: object(3)
memory usage: 170.8+ KB


In [ ]:
train.nunique() # 128개를 맞춰야하는 다중분류

id          16986
img_path    16986
overview    16973
cat1            6
cat2           18
cat3          128
dtype: int64

In [ ]:
test.nunique()

id          7280
img_path    7280
overview    7280
dtype: int64

In [ ]:
le = LabelEncoder() # 정수로 바꿔줌 
train["target"] = le.fit_transform(train['cat3'].to_numpy())
train

,id,img_path,overview,cat1,cat2,cat3,target
0,TRAIN_00000,./image/train/TRAIN_00000.jpg,소안항은 조용한 섬으로 인근해안이 청정해역으로 일찍이 김 양식을 해서 높은 소득을 ...,자연,자연관광지,항구/포구,120
1,TRAIN_00001,./image/train/TRAIN_00001.jpg,경기도 이천시 모가면에 있는 골프장으로 대중제 18홀이다. 회원제로 개장을 했다가 ...,레포츠,육상 레포츠,골프,8
2,TRAIN_00002,./image/train/TRAIN_00002.jpg,금오산성숯불갈비는 한우고기만을 전문적으로 취급하고 사용하는 부식 자재 또한 유기농법...,음식,음식점,한식,118
3,TRAIN_00003,./image/train/TRAIN_00003.jpg,철판 위에서 요리하는 안동찜닭을 맛볼 수 있는 곳이다. 경상북도 안동시에 있는 한식...,음식,음식점,한식,118
4,TRAIN_00004,./image/train/TRAIN_00004.jpg,※ 영업시간 10:30 ~ 20:30\n\n3대에 걸쳐 아귀만을 전문으로 취급하는 ...,음식,음식점,한식,118
...,...,...,...,...,...,...,...
16981,TRAIN_16981,./image/train/TRAIN_16981.jpg,해발 12000m에 자리한 식담겸 카페점문점이다.<br>곤드레밥과 감자전을 판매하고...,음식,음식점,한식,118
16982,TRAIN_16982,./image/train/TRAIN_16982.jpg,설악힐호텔은 동해고속도로 속초톨게이트에서 멀지 않은 관광로 변에 있다. 속초의 대표...,숙박,숙박시설,모텔,31
16983,TRAIN_16983,./image/train/TRAIN_16983.jpg,충남 서산시 중심가에 위치한 줌모텔은 프라이버스가 보장되는 조용한 공간으로 가치가 ...,숙박,숙박시설,모텔,31
16984,TRAIN_16984,./image/train/TRAIN_16984.jpg,토토큰바위캠핑장은 경기도 가평지역 내에서도 청정지역으로 손꼽히는 지역으로 주변에 화...,레포츠,육상 레포츠,"야영장,오토캠핑장",73


In [ ]:
!pip install kiwipiepy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from kiwipiepy import Kiwi

kiwi = Kiwi()
kiwi.prepare()

In [ ]:
lst = train["overview"].tolist()
gen = kiwi.analyze(lst)
train_overview = []

for tokens in tqdm(gen,total=len(lst)):
    token = [ token.form for token in tokens[0][0] if token.tag[0] in "NV" ]
    train_overview.append(" ".join(token)) # 띄어쓰기 기준으로 단어를 나눔

lst = test["overview"].tolist()
gen = kiwi.analyze(lst)
test_overview = []
for tokens in tqdm(gen,total=len(lst)):
    token = [ token.form for token in tokens[0][0] if token.tag[0] in "NV" ]
    test_overview.append(" ".join(token))

  0%|          | 0/16986 [00:00<?, ?it/s]

  0%|          | 0/7280 [00:00<?, ?it/s]

In [ ]:
train["overview_clean"] = train_overview # to .csv저장해서 빨리
test["overview_clean"] = test_overview

In [ ]:
max_features=4096
tfidf_vec = TfidfVectorizer(max_features=max_features)
train_vec = tfidf_vec.fit_transform(train["overview_clean"]).toarray()
test_vec = tfidf_vec.transform(test["overview_clean"]).toarray()

In [ ]:
mins = train_vec.min(axis=0)  # 민맥스 스케일링
sizes = train_vec.max(axis=0) - mins 
train_vec = (train_vec - mins) / sizes 
test_vec = (test_vec - mins) / sizes 
train_vec.shape , test_vec.shape

((16986, 4096), (7280, 4096))

In [ ]:
train_imgs = train["img_path"].to_numpy() # 이미지파일을 넘파이어레이로 바꿈
test_imgs = test["img_path"].to_numpy()
train_imgs.shape , test_imgs.shape

((16986,), (7280,))

In [ ]:
target = train["target"].to_numpy()
target.shape

(16986,)

In [ ]:
from torchvision import transforms


img_size = [224,224]

train_lst = [
    transforms.Resize(img_size),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]
train_transform = transforms.Compose(train_lst)

test_lst = [
    transforms.Resize(img_size),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]
test_transform = transforms.Compose(test_lst)

In [ ]:
class MyDataset(torch.utils.data.Dataset):
    def __init__(self , transform , img_path, vec, y = None ): 
        self.transform = transform
        self.img_path = img_path
        self.vec = vec
        self.y = y
    def __len__(self): 
        return self.img_path.shape[0]
    def __getitem__(self, idx): 
        item = {}
        x = Image.open(self.img_path[idx])
        item["imgs"] = self.transform(x)
        item["vec"] = torch.Tensor(self.vec[idx])
        if self.y is not None:
            item["y"] = torch.tensor(self.y[idx])
        return item   

In [ ]:
dt = MyDataset(train_transform,train_imgs,train_vec,target)
dl = torch.utils.data.DataLoader(dt, batch_size = 2 , shuffle=False)
batch = next(iter(dl))
batch

{'imgs': tensor([[[[-0.7479, -0.7479, -0.7650,  ..., -1.4500, -1.4329, -1.4500],
           [-0.7479, -0.7479, -0.7650,  ..., -1.4672, -1.4500, -1.4329],
           [-0.7479, -0.7650, -0.7479,  ..., -1.4500, -1.4672, -1.4500],
           ...,
           [ 0.2111, -0.1486, -0.4397,  ..., -0.5424, -0.5596, -0.5424],
           [-0.4568, -0.6794, -0.7137,  ..., -0.5767, -0.5767, -0.5424],
           [-0.7479, -0.7650, -0.7650,  ..., -0.5767, -0.5596, -0.5596]],
 
          [[-0.5126, -0.5126, -0.5301,  ..., -1.0553, -1.0553, -1.0728],
           [-0.5126, -0.5126, -0.5301,  ..., -1.0553, -1.0378, -1.0553],
           [-0.5126, -0.5301, -0.5126,  ..., -1.0378, -1.0378, -1.0378],
           ...,
           [ 0.0476, -0.3025, -0.5826,  ..., -0.6702, -0.7052, -0.6877],
           [-0.6176, -0.7927, -0.8277,  ..., -0.6702, -0.7052, -0.7052],
           [-0.8803, -0.8452, -0.8277,  ..., -0.7052, -0.6877, -0.7227]],
 
          [[ 0.0256,  0.0256,  0.0431,  ..., -0.2010, -0.2184, -0.2532],
     

In [ ]:
from torchvision.models import resnet50 , ResNet50_Weights 

In [ ]:
train.target.nunique()

128

In [ ]:
class Net(torch.nn.Module):
    def __init__(self,max_features): 
        super().__init__()
        self.pre_model = resnet50(weights=ResNet50_Weights.IMAGENET1K_V2)
        self.pre_model.fc = torch.nn.Linear(2048,1024)

        self.seq = torch.nn.Sequential(
            torch.nn.Linear(max_features,max_features//2),
            torch.nn.ReLU(),
            torch.nn.Linear(max_features//2,max_features//4),
            torch.nn.ReLU(),
        )
        self.output = torch.nn.Linear(max_features//4 + 1024,128)

    def forward(self, x1,x2):
        x1 = self.pre_model(x1)
        x2 = self.seq(x2)
        x = torch.cat([x1,x2],dim=1)
        x = self.output(x)
        return x

In [ ]:
!pip install torchinfo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import torchinfo
model = Net(max_features)
torchinfo.summary(model,[(32,3,224,224),(32,max_features)])  

Layer (type:depth-idx)                        Output Shape              Param #
Net                                           [32, 128]                 --
├─ResNet: 1-1                                 [32, 1024]                --
│    └─Conv2d: 2-1                            [32, 64, 112, 112]        9,408
│    └─BatchNorm2d: 2-2                       [32, 64, 112, 112]        128
│    └─ReLU: 2-3                              [32, 64, 112, 112]        --
│    └─MaxPool2d: 2-4                         [32, 64, 56, 56]          --
│    └─Sequential: 2-5                        [32, 256, 56, 56]         --
│    │    └─Bottleneck: 3-1                   [32, 256, 56, 56]         75,008
│    │    └─Bottleneck: 3-2                   [32, 256, 56, 56]         70,400
│    │    └─Bottleneck: 3-3                   [32, 256, 56, 56]         70,400
│    └─Sequential: 2-6                        [32, 512, 28, 28]         --
│    │    └─Bottleneck: 3-4                   [32, 512, 28, 28]         379,392

In [ ]:
def train_loop(dataloader,model,loss_fn,optimizer,device):
    epoch_loss = 0 
    model.train() 
    for batch in tqdm(dataloader): 
        pred = model(batch["imgs"].to(device),batch["vec"].to(device)) #이미지 텐서와 벡터 텐서 두개를 받기때문에 x1,x2
        loss = loss_fn(pred, batch["y"].to(device)) 
        
        optimizer.zero_grad() 
        loss.backward()  
        optimizer.step() 
        
        epoch_loss += loss.item() 

    epoch_loss /= len(dataloader) 

    return epoch_loss 

In [ ]:
@torch.no_grad() 
def test_loop(dataloader,model,loss_fn,device): 
    epoch_loss = 0
    model.eval() 

    
    pred_list = []
    softmax = torch.nn.Softmax(dim=1) 

    for batch in tqdm(dataloader):
        
        pred = model(batch["imgs"].to(device),batch["vec"].to(device))
        if batch.get("y") is not None: 
            loss = loss_fn(pred, batch["y"].to(device))
            epoch_loss += loss.item()
        
        pred = softmax(pred)
        pred = pred.to("cpu").numpy() 
        pred_list.append(pred)

    epoch_loss /= len(dataloader)

    pred = np.concatenate(pred_list) 
    return epoch_loss , pred 

In [ ]:
n_splits = 5
batch_size = 48 
epochs = 100
loss_fn = torch.nn.CrossEntropyLoss() # 다중분류 손실객체

In [ ]:
from sklearn.metrics import f1_score
from sklearn.model_selection import KFold
cv = KFold(n_splits=n_splits,shuffle=True, random_state=SEED)

In [ ]:
is_holdout = True
seed_everything(SEED)
best_score_list = []
for i,(tri,vai) in enumerate(cv.split(train_imgs)):
    
    model = Net(max_features).to(device)
    optimizer = torch.optim.Adam(model.parameters()) 

    train_dt = MyDataset(train_transform,train_imgs[tri],train_vec[tri],target[tri])
    valid_dt = MyDataset(test_transform,train_imgs[vai],train_vec[vai],target[vai])
    train_dl = torch.utils.data.DataLoader(train_dt, batch_size=batch_size, shuffle=True)
    valid_dl = torch.utils.data.DataLoader(valid_dt, batch_size=batch_size,shuffle=False)

    best_score = 0
    patience = 0

    for epoch in range(epochs):
        
        train_loss = train_loop(train_dl, model, loss_fn,optimizer,device )
        valid_loss , pred = test_loop(valid_dl, model, loss_fn,device  )
        
        pred = np.argmax(pred, axis=1) 
        true = target[vai] 
        score = f1_score(true, pred , average="weighted")
        print(train_loss,valid_loss,score)
        patience += 1
        if best_score < score:
            patience = 0
            best_score = score
            torch.save(model.state_dict(),f"{DATA_PATH}model_{i}.pth")

        if patience == 5:
            break
    print(f"Fold ({i}), BEST F1: {best_score}")
    best_score_list.append(best_score)

    torch.cuda.empty_cache()

    if is_holdout:
        break

  0%|          | 0/284 [00:00<?, ?it/s]

  0%|          | 0/71 [00:00<?, ?it/s]

1.7870212415574303 1.3044759513626636 0.6689903987578532


  0%|          | 0/284 [00:00<?, ?it/s]

  0%|          | 0/71 [00:00<?, ?it/s]

...

  0%|          | 0/71 [00:00<?, ?it/s]

0.030174334054016736 2.029659547436405 0.6988458836744924


  0%|          | 0/284 [00:00<?, ?it/s]

  0%|          | 0/71 [00:00<?, ?it/s]

0.041773545252639654 2.2416311712332173 0.6912662889847655
Fold (0), BEST F1: 0.7080150331568227


In [ ]:
np.mean(best_score_list)

0.7080150331568227

In [ ]:
test_dt = MyDataset(test_transform,test_imgs,test_vec)
test_dl = torch.utils.data.DataLoader(test_dt, batch_size=batch_size, shuffle=False)

pred_list = []
for i in range(n_splits):
    
    model = Net(max_features).to(device)
    state_dict = torch.load(f'{DATA_PATH}model_{i}.pth') 
    model.load_state_dict(state_dict)
        
    _ , pred = test_loop(test_dl, model, loss_fn,device  )
    pred_list.append(pred)
    
    if is_holdout:
        break

  0%|          | 0/152 [00:00<?, ?it/s]

In [ ]:
pred = np.mean(pred_list,axis=0) 
pred = np.argmax(pred, axis=1) 
pred

array([118, 118, 118, ...,  73, 119,  97])

In [ ]:
submit = pd.read_csv("sample_submission.csv")
submit["cat3"] = le.inverse_transform(pred)
submit

,id,cat3
0,TEST_00000,한식
1,TEST_00001,한식
2,TEST_00002,한식
3,TEST_00003,수련시설
4,TEST_00004,산
...,...,...
7275,TEST_07275,한식
7276,TEST_07276,공연장
7277,TEST_07277,"야영장,오토캠핑장"
7278,TEST_07278,한옥스테이


In [ ]:
submit.to_csv(f'{DATA_PATH}submit.csv', index=False)